In [1]:
import numpy as np
import lvlspy.spcoll as lc
import matplotlib.pyplot as plt

In [2]:
def transfer_properties(rate_matrix):
    n = len(rate_matrix)
    F = np.zeros([n-2,n-2]) #transfer matrix
    #transfer regarding 1
    f1_in = np.zeros(n-2)
    f1_out =np.zeros(n-2)
    #transfer regarding 2
    f2_in  = np.zeros(n-2)
    f2_out = np.zeros(n-2)

    #Calculating the rate matrix and transfer arrays
    for i in range(len(F)):
        f1_in[i] = rate_matrix[0,i+2]/rate_matrix[0,0]
        f1_out[i] = rate_matrix[i+2,0]/rate_matrix[i+2,i+2]

        f2_in[i] = rate_matrix[1,i+2]/rate_matrix[1,1]
        f2_out[i] = rate_matrix[i+2,1]/rate_matrix[i+2,i+2]
        for j in range(len(F)):
            F[i,j] = rate_matrix[i+2,j+2]/rate_matrix[i+2,i+2]
    

    return F,f1_in,f1_out,f2_in,f2_out

In [3]:
#setup the collection array to read the xml and load the al26 data
new_coll = lc.SpColl()
new_coll.update_from_xml('example.xml')

sp = new_coll.get()['al26']

#Temperature Range
T = np.linspace(0.1,10)*1e+9
lambda_21_eff = np.empty(len(T))
lambda_12_eff = np.empty(len(T))

for i,t in enumerate(T):
    rate_matrix = sp.compute_rate_matrix(t)
    F,f1_in,f1_out,f2_in,f2_out = transfer_properties(rate_matrix)
    
    #calculating lambda_21_eff
    dummy = np.matmul(np.linalg.inv(np.identity(len(F)) - F.T),f2_out)
    dummy = np.matmul(f1_in.T,dummy)
    
    lambda_21_eff[i] = dummy*rate_matrix[1,1]
    #calculating lambda_12_eff
    dummy = np.matmul(np.linalg.inv(np.identity(len(F)) -F.T),f1_out)
    dummy = np.matmul(f1_in.T,dummy)

    lambda_12_eff[i] = rate_matrix[0,0]*(dummy - 1.0)

ValueError: negative dimensions are not allowed

In [ ]:
plt.figure(figsize = (12,12))
fontsize = 40
plt.xscale('log')
plt.yscale('log')

plt.ylim([1.e-15,1.e+15])
plt.xlim([0.1,10])

plt.plot(T/1e+9,lambda_21_eff)
plt.ylabel(r'$\lambda_{21}^{eff} (s^{-1})$')
plt.xlabel(r'$T_{9}$')
plt.axhline(y = 1.e-1,ls = '--',color = 'black')
plt.show()